<a href="https://colab.research.google.com/github/guillaume-lrt/Covid19-retweet-prediction-challenge/blob/main/Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
!pip install verstack
from verstack.stratified_continuous_split import scsplit # pip install verstack

  Created wheel for verstack: filename=verstack-0.3.1-cp36-none-any.whl size=14343 sha256=6f85f282c2c668077ddb6a1443a7ae1165ae0af1f5661d2598230ed6cdbb00c2
  Stored in directory: /root/.cache/pip/wheels/15/1b/58/10e59516150cea4d9b1dbacceb3bffcc0cfd2d166efabec2f6
Successfully built verstack


In [12]:
train_data = pd.read_csv('https://media.githubusercontent.com/media/guillaume-lrt/Covid19-retweet-prediction-challenge/main/data/train.csv',error_bad_lines=False)
print(train_data)

            id  ...                                               text
0            0  ...                                      Smh I give up
1            1  ...  Most of us are Human Beings, but I think you m...
2            2  ...  Old dirty tricks Trump, at it again...like we ...
3            3  ...  Seriously..... I worked 86 hours my last check...
4            4  ...  May ALMIGHTY ALLAH have mercy on us all. Only ...
...        ...  ...                                                ...
665772  665772  ...                     18 months dawg? Come on man...
665773  665773  ...  Thank you to all of the nurses in our @Stanfor...
665774  665774  ...  'Post it' pearls for Palliative, End of Life a...
665775  665775  ...  His facial expressions are kind of looking for...
665776  665776  ...                              We really can't wait.

[665777 rows x 11 columns]


In [13]:
# Load the training data
# train_data = pd.read_csv("train.csv")
# train_data = pd.read_csv("https://github.com/guillaume-lrt/Covid19-retweet-prediction-challenge/blob/main/data/evaluation.csv")

# Here we split our training data into trainig and testing set. This way we can estimate the evaluation of our model without uploading to Kaggle and avoid overfitting over our evaluation dataset.
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweet_count'], stratify=train_data['retweet_count'], train_size=0.7, test_size=0.3)

# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweet_count'], axis=1)
X_test = X_test.drop(['retweet_count'], axis=1)

# You can examine the available features using X_train.head()

# We set up an Tfidf Vectorizer that will use the top 100 tokens from the tweets. We also remove stopwords.
# To do that we have to fit our training dataset and then transform both the training and testing dataset. 
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
X_train = vectorizer.fit_transform(X_train['text'])
X_test = vectorizer.transform(X_test['text'])

# Now we can train our model. Here we chose a Gradient Boosting Regressor and we set our loss function 
reg = GradientBoostingRegressor()
# We fit our model using the training data
reg.fit(X_train, y_train)
# And then we predict the values for our testing set
y_pred = reg.predict(X_test)
print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))

Prediction error: 261.8316245537317


In [15]:
###################################
# Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
###################################
# Load the evaluation data
eval_data = pd.read_csv("https://media.githubusercontent.com/media/guillaume-lrt/Covid19-retweet-prediction-challenge/main/data/train.csv",error_bad_lines=False)
# Transform our data into tfidf vectors
vectorizer = TfidfVectorizer(max_features=100, stop_words='english')
y_train = train_data['retweet_count']
X_train = vectorizer.fit_transform(train_data['text'])
# We fit our model using the training data
reg = GradientBoostingRegressor()
reg.fit(X_train, y_train)
X_val = vectorizer.transform(eval_data['text'])
# Predict the number of retweets for the evaluation dataset
y_pred = reg.predict(X_val)

In [ ]:
# Dump the results into a file that follows the required Kaggle template
with open("gbr_predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])
    for index, prediction in enumerate(y_pred):
        writer.writerow([str(eval_data['id'].iloc[index]) , str(int(prediction))])